
# Лабораторная работа 3. Вывод для высокоразмерных и неструктурированных данных. Оценка доверия: bootstrap, permutation, cross-validation. Интерпретация: feature importance, SHAP, causal inference.


**Курс:** Прикладная статистика и анализ данных

**Раздел 3:** Авангардные подходы для ИИ и доверие к моделям    

**Цель:** Построить полный цикл надёжного статистического моделирования: от корректного выбора протокола кросс-валидации и конфигурации модели до интерпретации результатов (feature importance/SHAP), причинной аргументации (DAG, эффекты вмешательств) и репликации внешних результатов в воспроизводимой среде.




## 1. Данные и дизайн эксперимента
Набор данных представляет собой наблюдения вида $(x_i, y_i)$, где $x_i$ — вектор признаков (количественных и/или категориальных), а $y_i$ — целевая переменная (например, числовой показатель эффективности, выручки или другого экономического/технического результата). Предполагается, что данные получены из стационарного источника и могут рассматриваться как реализация выборки из некоторого (неизвестного) распределения. На этапе предобработки выполняются базовые шаги: удаление или обработка пропусков, кодирование категориальных переменных, при необходимости масштабирование числовых признаков, а также выделение обучающей и тестовой подвыборок.

Цель эксперимента — не только построить предсказательную модель, но и провести статистический вывод относительно её характеристик и структуры. Для этого в работе используется несколько классов моделей (как минимум одна «прозрачная» базовая модель и одна более сложная модель), обучаемых на обучающей выборке. Тестовая выборка откладывается и используется для оценки обобщающей способности моделей и для последующей интерпретации результатов.

Дизайн эксперимента включает три ключевых компонента. Во-первых, для оценки неопределённости метрик качества и параметров моделей применяется бутстреп по объектам: из исходной обучающей выборки многократно формируются бутстреп-выборки, на каждой из которых переобучается модель и вычисляются интересующие статистики (например, $R^2$, MSE, разности ошибок моделей). Полученное эмпирическое распределение статистики используется для построения доверительных интервалов. Во-вторых, для проверки гипотез о значимости различий между моделями и/или признаками применяются перестановочные тесты, основанные на многократной перестановке меток или отдельных признаков и сравнении наблюдаемой статистики с её нулевым распределением. В-третьих, для анализа структуры модели и вклада отдельных признаков используются модель-агностические методы интерпретации: перестановочная важность признаков и SHAP-значения. Перестановочная важность вычисляется как изменение метрики качества при искусственном «разрушении» связи признака с целевой переменной, тогда как SHAP-значения позволяют получить аддитивное разложение предсказаний на уровне отдельных наблюдений. Совокупно такой дизайн эксперимента обеспечивает не только оценку качества моделей, но и количественно обоснованную интерпретацию полученных результатов.


## 2. Статистический вывод и доверительные интервалы в высокоразмерных задачах

Статистический вывод в контексте высокоразмерных и неструктурированных данных нацелен на переход от наблюдаемой выборки к утверждениям о генеративном механизме данных. Пусть наблюдается выборка $Y_1,\dots,Y_n$ с признаками $X_1,\dots,X_n$, а интересующий параметр обозначен как $\theta$ (например, коэффициент регрессии, средний эффект вмешательства или разность предсказательной ошибки двух моделей). Точечная оценка $\hat\theta$ сама по себе малоинформативна без понимания её неопределённости. Поэтому ключевыми объектами статистического вывода являются доверительные интервалы для $\theta$ и процедуры проверки гипотез вида $H_0:\theta=\theta_0$ против альтернатив $H_1$.

В классической низкоразмерной ситуации неопределённость оценки выводится из теоретического распределения статистики. Например, для линейной регрессии при стандартных предположениях используется асимптотическая нормальность оценок МНК и формула доверительного интервала
$
CI_{1-\alpha}(\theta)=\hat\theta \pm z_{1-\alpha/2},\widehat{\mathrm{se}}(\hat\theta),
$
где $\widehat{\mathrm{se}}(\hat\theta)$ — оценка стандартной ошибки, а $z_{1-\alpha/2}$ — квантиль стандартного нормального распределения. В высокоразмерных задачах, с большим числом признаков и сложными моделями (градиентный бустинг, случайные леса, нейросети), аналитическое выражение для распределения $\hat\theta$ либо отсутствует, либо опирается на заведомо нарушенные предположения (нормальность ошибок, гомоскедастичность, независимость наблюдений).

Дополнительная сложность связана с тем, что нас обычно интересуют не только «параметры» внутри модели, но и функционалы от модели: предсказательная ошибка $Q(f)$ на новых данных, разница ошибок $\Delta Q$ двух моделей, частные эффекты признаков, меры справедливости и устойчивости. Все эти величины представляют собой статистики $T(Y,X)$ со сложной зависимостью от данных. Для них традиционные формулы стандартных ошибок часто просто отсутствуют.

Современный подход к выводу в таких условиях опирается на аппроксимацию выборочного распределения статистики с помощью методов ресемплинга. Основная идея состоит в том, чтобы искусственно сгенерировать множество «альтернативных выборок», совместимых с наблюдаемыми данными и нулевой гипотезой, и по ним эмпирически оценить распределение $T$. На этой логике основаны бутстреп-процедуры и перестановочные тесты, которые позволяют строить доверительные интервалы и p-значения без жёсткой привязки к параметрическим предположениям о распределении ошибок.

Наконец, в высокоразмерных задачах особую роль играют вопросы воспроизводимости. При оценке доверительных интервалов и p-значений важно фиксировать генераторы случайных чисел, сохранять индексы ресемплированных наблюдений и чётко документировать выбранные настройки (число репликаций, тип интервалов, способ формирования статистики). Это позволяет повторно получить те же выводы, сравнивать их при изменении модели и использовать результаты как часть формальной доказательной базы при принятии решений.

## 3. Бутстреп и перестановочные тесты: оценка неопределённости без сильных предположений

Бутстреп — это общий подход к оценке неопределённости статистики $T(Y,X)$ посредством многократного ресемплинга из наблюдаемой выборки. В простейшей непараметрической версии бутстрепа предполагается, что эмпирическое распределение данных достаточно хорошо аппроксимирует истинное распределение. Тогда, вместо работы с теоретическим распределением, мы многократно генерируем бутстреп-выборки $Y_1^\ast,\dots,Y_n^\ast$ путём выборки с возвращением из исходных наблюдений $(Y_1,\dots,Y_n)$. Для каждой репликации $b=1,\dots,B$ вычисляется статистика $T^{\ast}_b$, после чего эмпирическое распределение ${T^{\ast}b}{b=1}^B$ служит приближением выборочного распределения $T$.

На основе бутстреп-распределения легко строятся доверительные интервалы. Наиболее простой вариант — процентильный бутстреп: интервал уровня $1-\alpha$ задаётся как $[q_{\alpha/2},q_{1-\alpha/2}]$, где $q_{p}$ — $p$-квантиль эмпирического распределения ${T^{\ast}_b}$. Более продвинутые варианты (BC, BCa) дополнительно корректируют смещение и асимметрию распределения. Бутстреп удобно применять для сложных статистик: разности ошибок двух моделей, медианы, коэффициентов в регуляризованных регрессиях, показателей справедливости.

Перестановочные тесты (randomization / permutation tests) решают родственную, но концептуально отличную задачу. Здесь формируется нулевая гипотеза о том, что метки или группы в данных взаимозаменяемы (exchangeable) при $H_0$. Например, при сравнении двух групп предполагается, что распределения в группах совпадают, и различие средних обусловлено лишь случайной разметкой наблюдений. Тестовая статистика $T_{\text{obs}}$ (разность средних, статистика $t$, разность ошибок моделей и т.п.) вычисляется на исходных данных. Далее значения меток многократно случайно переставляются, при каждой перестановке пересчитывается $T^{\pi}_b$, и распределение ${T^{\pi}_b}$ используется как референс-распределение при $H_0$. p-значение оценивается как доля перестановок с $|T^{\pi}b|\ge|T{\text{obs}}|$.

Принципиальное различие между бутстрепом и перестановочными тестами состоит в том, что бутстреп нацелен на оценку параметров и их неопределённости, тогда как перестановка фокусируется на проверке гипотез о распределениях и структуре связи (например, наличии систематического эффекта модели или вмешательства). В практических задачах ЛР3 оба подхода используются совместно: бутстреп позволяет строить доверительные интервалы для метрик качества и важности признаков, а перестановочные тесты — проверять значимость различий между моделями и устойчивость выявленных связей при минимуме параметрических допущений.

## 4. Важность признаков и глобальная интерпретируемость сложных моделей

При работе с высокоразмерными данными и сложными моделями $f(x)$ (ансамбли деревьев, бустинг, нейросети) ключевым вопросом становится оценка вклада отдельных признаков в предсказание. Под важностью признака в глобальном смысле обычно понимают меру того, насколько использование данного признака улучшает некоторый показатель качества модели, усреднённый по распределению входных данных. Глобальная интерпретируемость отвечает на вопрос «какие признаки в среднем наиболее влиятельны для предсказаний на всей выборке», в отличие от локальных методов, анализирующих одно конкретное наблюдение.

В простых моделях (линейная регрессия, логистическая регрессия) в роли меры важности часто выступают стандартизованные коэффициенты, отношения шансов или t-статистики. Однако в нелинейных моделях такие прямые интерпретации отсутствуют, а сами параметры не имеют однозначного смыслового сопоставления между моделями. Поэтому используются специальные меры важности: основанные на структуре модели (например, сумма уменьшений импьюрити в деревьях решений) и модель-агностические. Модель-агностические подходы определяют важность через изменение предсказательной ошибки при искусственном «выключении» или искажении признака.

Наиболее распространённый модель-агностический подход — перестановочная важность (permutation importance). Для признака $j$ строится модифицированный набор $X^{\pi(j)}$, в котором значения $x_j$ случайно переставлены по наблюдениям, разрушая связь этого признака с целевой переменной, но сохраняя его маргинальное распределение. Модель $f$ применяют к $X$ и $X^{\pi(j)}$, после чего важность определяют как
$
\mathrm{Imp}(j) = Q(f;X,y) - Q(f;X^{\pi(j)},y),
$
где $Q$ — выбранная метрика качества (например, $R^2$, AUC, лог-лосс). Чем сильнее ухудшается качество при перестановке, тем более важен признак. Повторение процедуры с несколькими случайными перестановками позволяет оценить стандартизированную важность и доверительные интервалы.

Перестановочная важность удобна тем, что применима к любой «чёрной коробке» и может быть одинаково реализована для деревьев, бустинга и нейросетей. Однако у метода есть ограничения: при сильной корреляции признаков их важность может «размазываться» между группой связанных признаков; возможно появление отрицательной важности при переобучении или некорректной оценке качества. Поэтому в ЛР3 важно не только вычислить важности, но и сопоставить их с предметным смыслом признаков, проверить устойчивость результатов к изменению метрики и способа разбиения выборки, а также при необходимости применять групповые перестановки для логически связанных наборов признаков.

## 5. SHAP-значения и локальная интерпретация предсказаний

Метод SHAP (SHapley Additive exPlanations) основан на переносе идей кооперативной теории игр в контекст интерпретации моделей машинного обучения. Рассматривается множество признаков $F={1,\dots,p}$ как набор «игроков», совместно формирующих «стоимость» — предсказание модели $f(x)$. Для каждого признака $j$ вводится его вклад $\phi_j(x)$ в предсказание для конкретного наблюдения $x$. Идея состоит в том, чтобы определять $\phi_j(x)$ как усреднённый маргинальный вклад признака $j$ при присоединении его к всевозможным подмножествам $S\subseteq F\setminus{j}$:

$
\phi_j(x)=\sum_{S\subseteq F\setminus{j}} \frac{|S|!,(|F|-|S|-1)!}{|F|!} \left[v_x(S\cup{j})-v_x(S)\right],
$

где $v_x(S)$ — «значение коалиции» $S$, интерпретируемое как ожидаемое предсказание модели при известны только признаки из $S$, а остальные усреднены по их распределению. Такое определение обеспечивает несколько важных свойств: локальную аддитивность (сумма $\sum_j \phi_j(x)$ восстанавливает сдвиг между базовым уровнем и предсказанием для $x$), симметрию (равнозначные признаки имеют одинаковые вклады) и согласованность (если модель меняется так, что маргинальный вклад признака растёт во всех контекстах, его SHAP-важность не уменьшается).

На практике точное вычисление Шеплиевских значений по формуле выше требует суммирования по всем подмножествам признаков и потому экспоненциально сложно по $p$. Однако для важных классов моделей (в первую очередь деревьев решений и их ансамблей) разработаны эффективные алгоритмы, реализованные, например, в библиотеке SHAP. Эти алгоритмы позволяют вычислять SHAP-значения для каждого наблюдения и признака с полиномиальной сложностью, что делает метод применимым в задачах реального масштаба.

Результаты SHAP можно интерпретировать как локальные объяснения предсказаний: для каждого конкретного объекта становится видно, какие признаки «подтягивают» предсказание вверх, а какие — вниз и в каком численном объёме. Агрегируя $|\phi_j(x_i)|$ по выборке, получают глобальные меры важности признаков, сопоставимые с перестановочной важностью, но обладающие более чёткой теоретической интерпретацией. SHAP-диаграммы (summary plots, dependence plots) позволяют обнаруживать нелинейности, пороговые эффекты и взаимодействия признаков, которые не видны из одних только коэффициентов или глобальных важностей.

При этом важно помнить, что SHAP описывает предсказательную модель, а не причинные связи в данных. Высокое по модулю значение $\phi_j(x)$ означает сильное влияние признака на предсказание $f(x)$ при зафиксированном распределении других признаков, но не гарантирует существование причинного эффекта признака на целевую переменную. Поэтому в ЛР3 следует использовать SHAP совместно с предметным анализом, диаграммами причинно-следственных связей и, при необходимости, специальными методами причинного вывода для наблюдательных данных.

## 35 вариантов заданий (обобщающие, с привязкой к D1–D10)

### Общие методические требования (для всех вариантов):
1) Формулировка цели и базовой модели.
Для выбранного датасета чётко сформулируйте исследовательский вопрос (что именно прогнозируем/объясняем и зачем) и определите целевую переменную. Постройте базовую модель (часто — линейная регрессия или логистическая регрессия) с ограниченным числом очевидных предикторов. Зафиксируйте набор метрик качества (например, $R^2$, RMSE/MAE для регрессии или ROC-AUC/PR-AUC для классификации), которые будут использоваться в дальнейшем для сравнения моделей.

2) Дизайн кросс-валидации и выбор модели.
Разработайте протокол кросс-валидации, учитывая природу данных: для i.i.d.-наблюдений — стандартный $K$-fold с стратификацией по целевой переменной (в задачах дисбаланса); для временных рядов и логов — разбиения по времени (rolling/expanding window, TimeSeriesSplit). Обоснуйте выбор числа фолдов, размера обучающего/валидационного окна и схемы повторов. На основе CV подберите гиперпараметры основных моделей (например, коэффициент регуляризации $\lambda$ в Ridge/Lasso, глубину/число деревьев в бустинге), а также сравните несколько конкурирующих спецификаций.

3) Диагностика и интерпретируемость модели.
Для выбранной «рабочей» модели выполните анализ важности признаков: пермутационная важность, коэффициенты линейной модели, а также, при возможности, SHAP-значения для глобальной и локальной интерпретации. Сопоставьте результаты разных методов, выделите устойчивое ядро важных факторов и обсудите, насколько они согласуются с доменной логикой. Избегайте прямолинейного причинного толкования: зафиксируйте, что модель в первую очередь предиктивная, и аккуратно отделяйте интерпретацию «ассоциации» от «эффекта».

4) Причинно-следственные допущения и сценарии «что-если».
Даже если цель работы не в строгой идентификации причинного эффекта, постройте качественную DAG-диаграмму (граф зависимостей) для ключевых переменных, обозначьте предполагаемые конфаундеры, возможные промежуточные и обратные связи. Сформулируйте потенциальный причинный вопрос (например, «как изменится $Y$ при изменении некоторого управляемого фактора $X$?») и обсудите, какие дополнительные данные или экспериментальный дизайн потребовались бы для надёжного вывода. Явно опишите, какие оценки в вашей работе можно трактовать лишь как ассоциативные.

5) Оценка неопределённости и устойчивости выводов.
Для выбранных моделей и ключевых показателей качества примените бутстреп-процедуры (percentile/BCa) для получения доверительных интервалов для $R^2$, RMSE/MAE или других метрик, а также перестановочные тесты для оценки статистической значимости разницы в качестве между двумя моделями или спецификациями (например, «модель с дополнительными признаками vs. базовая»). В явном виде интерпретируйте интервал и $p$-значение, связывая их с практической значимостью (бизнес, транспорт, медицина и т.д.).

6) Репликация и воспроизводимость.
Обеспечьте воспроизводимость экспериментов:
- зафиксируйте случайные зерна, список версий библиотек и основные параметры среды;
- реализуйте весь препроцессинг, разбиения на выборки и обучение моделей внутри единого конвейера (например, Pipeline/ColumnTransformer в scikit-learn или явный код с чёткой структурой);
- документируйте шаги: от загрузки данных и фильтраций до построения финальных таблиц и рисунков;
- сформируйте краткий протокол репликации (какие файлы/скрипты и в каком порядке запускать, какие внешние источники данных используются).
При работе с публичными датасетами (Kaggle, Inside Airbnb и др.) обязательно фиксируйте версию выгрузки (дата скачивания, номер релиза) и, по возможности, сравнивайте свои результаты с опубликованными решениями или статьями, обсуждая причины возможных расхождений.


### Вариант
Вариант 1. NYC-FareCV (D1)
Предсказать итоговую стоимость поездки (total_amount или fare_amount) по данным NYC TLC Yellow Taxi. Сформируйте выборку поездок за один месяц и случайную подвыборку 100 000 наблюдений. Постройте ряд моделей (линейная регрессия, градиентный бустинг, случайный лес) и спроектируйте протокол CV, учитывая неоднородность по времени суток и районам. Сравните K-fold и временную кросс-валидацию (TimeSeriesSplit): оцените различия в оценках RMSE/MAE и прокомментируйте возможное смещение. Затем проведите анализ важности признаков (пермутационная важность и SHAP) для лучшей модели, выделите ключевые факторы тарифа (расстояние, район, час, тип оплаты). Сформулируйте причинную гипотезу об эффекте «час пик/не час пик» на стоимость за милю и постройте упрощённый DAG; обсудите, какие переменные следует контролировать. На этапе репликации выберите внешнее исследование/бенчмарк по NYC Taxi и попытайтесь воспроизвести сопоставимую метрику качества, фиксируя версии библиотек и протокол CV.


# Предобработка данных

In [1]:
import pandas as pd
import numpy as np

# Загружаем данные
df = pd.read_parquet("yellow_tripdata_2025-08.parquet")

print("Исходный размер:", df.shape)
df.head()

Исходный размер: (3574091, 20)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,2,2025-08-01 00:52:23,2025-08-01 01:12:20,1.0,8.44,1.0,N,138,141,1,33.8,6.0,0.5,5.00,6.94,1.0,57.49,2.5,1.75,0.00
1,2,2025-08-01 00:03:01,2025-08-01 00:15:33,2.0,4.98,1.0,N,138,193,1,21.2,6.0,0.5,0.00,0.00,1.0,30.45,0.0,1.75,0.00
2,7,2025-08-01 00:24:38,2025-08-01 00:24:38,2.0,1.89,1.0,N,249,45,1,14.2,0.0,0.5,3.99,0.00,1.0,23.94,2.5,0.00,0.75
3,7,2025-08-01 00:48:19,2025-08-01 00:48:19,1.0,2.35,1.0,N,79,229,1,11.4,0.0,0.5,3.43,0.00,1.0,20.58,2.5,0.00,0.75
4,2,2025-08-01 00:25:34,2025-08-01 00:33:18,1.0,2.14,1.0,N,43,48,1,11.4,1.0,0.5,2.57,0.00,1.0,19.72,2.5,0.00,0.75


In [2]:
# ----- 1) Оставляем нужные для модели столбцы -----

columns_needed = [
    "tpep_pickup_datetime", "tpep_dropoff_datetime",
    "passenger_count", "trip_distance",
    "PULocationID", "DOLocationID",
    "payment_type", "total_amount"
]

df = df[columns_needed].copy()

# ----- 2) Удаляем строки с NaN по важным признакам -----
df = df.dropna(subset=["passenger_count", "trip_distance", "total_amount"])

# ----- 3) Удаляем невозможные значения -----
df = df[(df["trip_distance"] > 0) & (df["total_amount"] > 0)]
df = df[df["trip_distance"] < 200]       # Физический максимум поездки в NYC
df = df[df["total_amount"] < 500]        # Защита от аномально дорогих поездок

print("Размер после очистки:", df.shape)
df.head()


Размер после очистки: (2574569, 8)


,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,total_amount
0,2025-08-01 00:52:23,2025-08-01 01:12:20,1.0,8.44,138,141,1,57.49
1,2025-08-01 00:03:01,2025-08-01 00:15:33,2.0,4.98,138,193,1,30.45
2,2025-08-01 00:24:38,2025-08-01 00:24:38,2.0,1.89,249,45,1,23.94
3,2025-08-01 00:48:19,2025-08-01 00:48:19,1.0,2.35,79,229,1,20.58
4,2025-08-01 00:25:34,2025-08-01 00:33:18,1.0,2.14,43,48,1,19.72


In [3]:
# ===== Генерация признаков =====

# 1) Час поездки
df["pickup_hour"] = df["tpep_pickup_datetime"].dt.hour

# 2) Продолжительность поездки в минутах
df["trip_duration_min"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60
df["trip_duration_min"] = df["trip_duration_min"].clip(lower=1)

# 3) Убираем datetime после извлечения признаков
df = df.drop(columns=["tpep_pickup_datetime", "tpep_dropoff_datetime"])

# 4) Бинning для районов по LocationID
df["PULocation_bin"] = pd.cut(df["PULocationID"], bins=20, labels=False)
df["DOLocation_bin"] = pd.cut(df["DOLocationID"], bins=20, labels=False)

# 5) Выбрасываем исходные LocationID
df = df.drop(columns=["PULocationID", "DOLocationID"])

print("Готово!")
df.head()


Готово!


,passenger_count,trip_distance,payment_type,total_amount,pickup_hour,trip_duration_min,PULocation_bin,DOLocation_bin
0,1.0,8.44,1,57.49,0,19.950000,10,10
1,2.0,4.98,1,30.45,0,12.533333,10,14
2,2.0,1.89,1,23.94,0,1.000000,18,3
3,1.0,2.35,1,20.58,0,1.000000,5,17
4,1.0,2.14,1,19.72,0,7.733333,3,3


In [4]:
# ===== Отделяем y =====
y = df["total_amount"].copy()
X = df.drop(columns=["total_amount"])

print("Размер X:", X.shape)
print("Размер y:", y.shape)

display(X.head())


Размер X: (2574569, 7)
Размер y: (2574569,)


,passenger_count,trip_distance,payment_type,pickup_hour,trip_duration_min,PULocation_bin,DOLocation_bin
0,1.0,8.44,1,0,19.950000,10,10
1,2.0,4.98,1,0,12.533333,10,14
2,2.0,1.89,1,0,1.000000,18,3
3,1.0,2.35,1,0,1.000000,5,17
4,1.0,2.14,1,0,7.733333,3,3


In [5]:
if len(df) > 100_000:
    df_sample = df.sample(100_000, random_state=42)
    y = df_sample["total_amount"]
    X = df_sample.drop(columns=["total_amount"])
    print("Взята подвыборка 100k:")
    print("Размер X:", X.shape)


Взята подвыборка 100k:
Размер X: (100000, 7)


# Предобработка (ColumnTransformer) + Модели (Pipeline)

In [6]:
# Проверим, какие признаки есть в X
print("Колонки X:", list(X.columns))

# Числовые признаки (будем масштабировать)
numeric_features = [
    "passenger_count",
    "trip_distance",
    "pickup_hour",
    "trip_duration_min"
]

# Категориальные признаки (будем one-hot кодировать)
categorical_features = [
    "payment_type",
    "PULocation_bin",
    "DOLocation_bin"
]

print("Числовые признаки:", numeric_features)
print("Категориальные признаки:", categorical_features)


Колонки X: ['passenger_count', 'trip_distance', 'payment_type', 'pickup_hour', 'trip_duration_min', 'PULocation_bin', 'DOLocation_bin']
Числовые признаки: ['passenger_count', 'trip_distance', 'pickup_hour', 'trip_duration_min']
Категориальные признаки: ['payment_type', 'PULocation_bin', 'DOLocation_bin']


In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def create_preprocessing_pipeline(numeric_features, categorical_features):
    """
    Создать пайплайн предобработки:
    - масштабирование числовых признаков
    - one-hot кодирование категориальных
    """

    numeric_transformer = Pipeline(steps=[
        ("scaler", StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features)
        ]
    )

    return preprocessor


def create_model_pipeline(preprocessor, model):
    """
    Создать полный пайплайн: предобработка + модель
    """
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", model)
    ])

    return pipeline


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# 1) Пайплайн предобработки
preprocessor = create_preprocessing_pipeline(numeric_features, categorical_features)

# 2) Модели
linear_model = LinearRegression()

rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=None,
    n_jobs=-1,
    random_state=42
)

gb_model = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

# 3) Полные пайплайны: предобработка + модель
pipeline_linear = create_model_pipeline(preprocessor, linear_model)
pipeline_rf = create_model_pipeline(preprocessor, rf_model)
pipeline_gb = create_model_pipeline(preprocessor, gb_model)

pipeline_linear, pipeline_rf, pipeline_gb


(Pipeline(steps=[('preprocessor',
                  ColumnTransformer(transformers=[('num',
                                                   Pipeline(steps=[('scaler',
                                                                    StandardScaler())]),
                                                   ['passenger_count',
                                                    'trip_distance',
                                                    'pickup_hour',
                                                    'trip_duration_min']),
                                                  ('cat',
                                                   Pipeline(steps=[('onehot',
                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                  sparse_output=False))]),
                                                   ['payment_type',
                                         

# Сравнение стратегий кросс-валидации

In [15]:
from sklearn.model_selection import KFold, TimeSeriesSplit, GroupKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

def compare_cv_strategies(X, y, pipeline, groups=None, n_splits=5):
    """
    Сравнить разные стратегии кросс-валидации для регрессии.
    
    Считаем RMSE и MAE для:
    - KFold (перемешанный)
    - TimeSeriesSplit (по порядку строк X, поэтому важно, чтобы X был отсортирован по времени)
    - GroupKFold (опционально, если передан groups)
    """

    cv_strategies = {
        "KFold": KFold(n_splits=n_splits, shuffle=True, random_state=42),
        "TimeSeriesSplit": TimeSeriesSplit(n_splits=n_splits)
    }

    # Добавляем GroupKFold, если заданы группы
    if groups is not None:
        cv_strategies["GroupKFold"] = GroupKFold(n_splits=n_splits)

    results = {}
    for cv_name, cv_splitter in cv_strategies.items():
        rmse_scores = []
        mae_scores = []

        # Особый случай для GroupKFold: нужен параметр groups
        if isinstance(cv_splitter, GroupKFold):
            split_iter = cv_splitter.split(X, y, groups=groups)
        else:
            split_iter = cv_splitter.split(X, y)

        for fold_idx, (train_idx, val_idx) in enumerate(split_iter, start=1):
            X_train_fold = X.iloc[train_idx]
            X_val_fold = X.iloc[val_idx]
            y_train_fold = y.iloc[train_idx]
            y_val_fold = y.iloc[val_idx]

            # Обучаем пайплайн
            pipeline.fit(X_train_fold, y_train_fold)

            # Предсказываем на валидации
            y_pred = pipeline.predict(X_val_fold)

            # --- ВАЖНО: считаем RMSE через sqrt(MSE), без параметра squared ---
            mse = mean_squared_error(y_val_fold, y_pred)
            rmse = np.sqrt(mse)

            mae = mean_absolute_error(y_val_fold, y_pred)

            rmse_scores.append(rmse)
            mae_scores.append(mae)
        results[cv_name] = {
            "rmse_mean": float(np.mean(rmse_scores)),
            "rmse_std": float(np.std(rmse_scores)),
            "mae_mean": float(np.mean(mae_scores)),
            "mae_std": float(np.std(mae_scores)),
        }

    return results


In [16]:
groups = X["PULocation_bin"]

cv_results_rf = compare_cv_strategies(
    X=X,
    y=y,
    pipeline=pipeline_rf,
    groups=groups,
    n_splits=5
)

cv_results_rf


{'KFold': {'rmse_mean': 7.333314532774233,
  'rmse_std': 0.3592797794238624,
  'mae_mean': 2.879975243812699,
  'mae_std': 0.04422039511813796},
 'TimeSeriesSplit': {'rmse_mean': 7.336057652308503,
  'rmse_std': 0.21414418073222985,
  'mae_mean': 2.936601710431989,
  'mae_std': 0.06804428284685471},
 'GroupKFold': {'rmse_mean': 8.215810598154256,
  'rmse_std': 3.07130001023099,
  'mae_mean': 3.5694660246023333,
  'mae_std': 1.7311874542656547}}

# Bootstrap доверительных интервалов

In [17]:
from sklearn.metrics import mean_squared_error
import numpy as np

def rmse_metric(y_true, y_pred):
    """Вычисление RMSE."""
    mse = mean_squared_error(y_true, y_pred)
    return float(np.sqrt(mse))


def bootstrap_confidence_interval(X, y, pipeline, metric_func, n_bootstrap=1000, alpha=0.05, random_state=42):
    """
    Построить доверительный интервал метрики через bootstrap.
    
    Параметры
    ---------
    X : pd.DataFrame
        Признаки (желательно train-часть).
    y : pd.Series
        Целевая переменная.
    pipeline : sklearn Pipeline
        Пайплайн (предобработка + модель).
    metric_func : callable
        Функция метрики: metric_func(y_true, y_pred) -> float.
        В нашем случае — RMSE.
    n_bootstrap : int
        Число bootstrap-итераций.
    alpha : float
        Уровень значимости (0.05 ⇒ 95% ДИ).
    random_state : int
        Зерно генератора случайных чисел.
    
    Возвращает
    ----------
    lower : float
        Нижняя граница доверительного интервала (percentile).
    upper : float
        Верхняя граница доверительного интервала.
    bootstrap_metrics : list of float
        Значения метрики на всех bootstrap-выборках.
    """
    n_samples = len(X)
    rng = np.random.default_rng(random_state)

    bootstrap_metrics = []

    for i in range(n_bootstrap):
        # 1) bootstrap-индексы с возвращением
        indices = rng.integers(0, n_samples, size=n_samples)

        X_boot = X.iloc[indices]
        y_boot = y.iloc[indices]

        # 2) обучение модели на bootstrap-выборке
        pipeline.fit(X_boot, y_boot)

        # 3) предсказание на этой же выборке
        y_pred = pipeline.predict(X_boot)

        # 4) вычисление метрики
        metric_value = metric_func(y_boot, y_pred)
        bootstrap_metrics.append(metric_value)

    # 5) percentiles для ДИ
    lower = float(np.percentile(bootstrap_metrics, 100 * alpha / 2))
    upper = float(np.percentile(bootstrap_metrics, 100 * (1 - alpha / 2)))

    return lower, upper, bootstrap_metrics


In [18]:
# Пример: bootstrap CI для RMSE RandomForest

lower_rmse, upper_rmse, boot_rmse_values = bootstrap_confidence_interval(
    X=X,          # в финальной версии лучше X_train
    y=y,          # и y_train
    pipeline=pipeline_rf,
    metric_func=rmse_metric,
    n_bootstrap=300,   # для начала можно взять 300, потом 1000
    alpha=0.05,
    random_state=42
)

print(f"95% bootstrap CI для RMSE (RandomForest): [{lower_rmse:.3f}, {upper_rmse:.3f}]")


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

diff_arr = np.array(differences_lr_rf)

plt.figure(figsize=(8, 5))
sns.histplot(diff_arr, bins=30, kde=True)
plt.axvline(0, linestyle="--")
plt.xlabel("Разность RMSE: Linear - RandomForest")
plt.ylabel("Частота")
plt.title("Bootstrap-распределение разности RMSE (Linear - RF)")
plt.tight_layout()
plt.show()

print("Средняя разность RMSE (Linear - RF):", diff_arr.mean())
print("Стандартное отклонение разности:", diff_arr.std())
print("Доля случаев, когда RF лучше (разность > 0):", (diff_arr > 0).mean())


# Сравнение моделей через bootstrap

In [ ]:
def compare_models_bootstrap(X, y, pipeline1, pipeline2, metric_func, 
                             n_bootstrap=1000, random_state=42):
    """
    Сравнить две модели через bootstrap по заданной метрике.
    
    На каждой bootstrap-итерации:
    - обучаем обе модели на одной и той же bootstrap-выборке;
    - считаем метрику для каждой;
    - сохраняем разность: metric1 - metric2.
    
    Параметры
    ---------
    X : pd.DataFrame
        Признаки (желательно train-часть).
    y : pd.Series
        Целевая переменная.
    pipeline1 : sklearn Pipeline
        Первая модель (например, LinearRegression).
    pipeline2 : sklearn Pipeline
        Вторая модель (например, RandomForestRegressor).
    metric_func : callable
        Функция метрики: metric_func(y_true, y_pred) -> float.
        В нашем случае — RMSE.
    n_bootstrap : int
        Количество bootstrap-итераций.
    random_state : int
        Зерно генератора случайных чисел.
    
    Возвращает
    ----------
    differences : list of float
        Список разностей metric1 - metric2 на каждой итерации.
        Если метрика = RMSE, то значения > 0 означают, что модель 2 лучше.
    """

    n_samples = len(X)
    rng = np.random.default_rng(random_state)

    differences = []

    for i in range(n_bootstrap):
        # bootstrap-индексы
        indices = rng.integers(0, n_samples, size=n_samples)

        X_boot = X.iloc[indices]
        y_boot = y.iloc[indices]

        # обучаем обе модели
        pipeline1.fit(X_boot, y_boot)
        pipeline2.fit(X_boot, y_boot)

        # предсказания
        y_pred1 = pipeline1.predict(X_boot)
        y_pred2 = pipeline2.predict(X_boot)

        # вычисляем метрики
        m1 = metric_func(y_boot, y_pred1)
        m2 = metric_func(y_boot, y_pred2)

        # разность: metric1 - metric2
        diff = m1 - m2
        differences.append(float(diff))

    return differences


In [ ]:
# Сравниваем LinearRegression (pipeline_linear) и RandomForest (pipeline_rf)
differences_lr_rf = compare_models_bootstrap(
    X=X,
    y=y,
    pipeline1=pipeline_linear,
    pipeline2=pipeline_rf,
    metric_func=rmse_metric,
    n_bootstrap=300,      # можно позже увеличить до 1000
    random_state=42
)

# Быстрая сводка
diff_arr = np.array(differences_lr_rf)
print("Средняя разность RMSE (Linear - RF):", diff_arr.mean())
print("Стандартное отклонение разности:", diff_arr.std())
print("Доля случаев, когда RF лучше (разность > 0):", (diff_arr > 0).mean())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

diff_arr = np.array(differences_lr_rf)

plt.figure(figsize=(8, 5))
sns.histplot(diff_arr, bins=30, kde=True)
plt.axvline(0, linestyle="--")
plt.xlabel("Разность RMSE: Linear - RandomForest")
plt.ylabel("Частота")
plt.title("Bootstrap-распределение разности RMSE (Linear - RF)")
plt.tight_layout()
plt.show()

print("Средняя разность RMSE (Linear - RF):", diff_arr.mean())
print("Стандартное отклонение разности:", diff_arr.std())
print("Доля случаев, когда RF лучше (разность > 0):", (diff_arr > 0).mean())


# Permutation-test значимости модели

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

def permutation_test(X, y, pipeline, metric_func, n_permutations=1000, random_state=42):
    """
    Permutation test значимости модели.
    
    1) Обучаем модель на исходных (X, y), считаем метрику (RMSE).
    2) n_permutations раз перемешиваем y, обучаем модель, считаем метрику.
    3) p-value = доля permuted_metric <= real_metric 
       (для RMSE: если модель хорошая, real_metric сильно меньше, p-value ~ 0).
    
    Параметры
    ---------
    X : pd.DataFrame
        Признаки.
    y : pd.Series
        Целевая.
    pipeline : sklearn Pipeline
        Модель (предобработка + алгоритм).
    metric_func : callable
        Функция метрики: metric_func(y_true, y_pred) -> float (RMSE).
    n_permutations : int
        Количество перестановок.
    random_state : int
        Зерно генератора случайных чисел.
    
    Возвращает
    ----------
    real_score : float
        Метрика на реальных данных.
    permuted_scores : list of float
        Метрики на перемешанных данных.
    p_value : float
        Оценка p-value.
    """

    rng = np.random.default_rng(random_state)

    # --- 1. Реальный скор ---
    pipeline.fit(X, y)
    y_pred = pipeline.predict(X)
    real_score = metric_func(y, y_pred)

    permuted_scores = []

    # Чтобы при перестановках не путаться с индексами, сделаем массив
    y_values = y.values

    for i in range(n_permutations):
        # Перестановка целевой
        perm_indices = rng.permutation(len(y_values))
        y_perm = y_values[perm_indices]

        # Обучаем на (X, y_perm)
        pipeline.fit(X, y_perm)
        y_pred_perm = pipeline.predict(X)

        perm_score = metric_func(y_perm, y_pred_perm)
        permuted_scores.append(float(perm_score))

    permuted_scores = np.array(permuted_scores)

    # --- 3. p-value ---
    # Для RMSE "меньше = лучше". Нулевая модель ожидается с БОЛЬШИМ RMSE.
    # Поэтому берём долю permuted_scores <= real_score
    p_value = float(np.mean(permuted_scores <= real_score))

    return float(real_score), permuted_scores, p_value


In [ ]:
# Используем уже определённую rmse_metric из предыдущих шагов

real_rmse_rf, perm_rmse_rf, p_value_rf = permutation_test(
    X=X,
    y=y,
    pipeline=pipeline_rf,
    metric_func=rmse_metric,
    n_permutations=300,   # чтобы не ждать очень долго, потом можно увеличить
    random_state=42
)

print(f"Real RMSE (RandomForest): {real_rmse_rf:.3f}")
print(f"Mean permuted RMSE: {perm_rmse_rf.mean():.3f}")
print(f"p-value: {p_value_rf:.4f}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.figure(figsize=(8, 5))
sns.histplot(perm_rmse_rf, bins=30, kde=True)
plt.axvline(real_rmse_rf, color="red", linestyle="--", label=f"Real RMSE = {real_rmse_rf:.2f}")
plt.xlabel("RMSE (перемешанный y)")
plt.ylabel("Частота")
plt.title("Permutation test для RandomForest")
plt.legend()
plt.tight_layout()
plt.show()


# Feature Importance

# SHAP анализ

In [ ]:
"""
Лабораторная работа 3: Скелет кода
Вывод для высокоразмерных данных, Bootstrap, CV, Feature Importance, SHAP
"""

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, TimeSeriesSplit, GroupKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
import shap
import matplotlib.pyplot as plt
import seaborn as sns

# ==============================================================================
# 1. ЗАГРУЗКА И ПРЕДВАРИТЕЛЬНАЯ ПОДГОТОВКА ДАННЫХ
# ==============================================================================

def load_and_prepare_data():
    """Загрузить датасет и провести базовую подготовку"""
    # TODO: Загрузите ваш датасет
    # df = pd.read_csv('your_dataset.csv')

    # TODO: Обработайте пропуски
    # df = df.dropna(subset=['target_column'])

    # TODO: Определите признаки и целевую переменную
    # X = df.drop('target', axis=1)
    # y = df['target']

    return None, None  # замените на X, y


# ==============================================================================
# 2. СОЗДАНИЕ ПАЙПЛАЙНОВ
# ==============================================================================

def create_preprocessing_pipeline(numeric_features, categorical_features):
    """Создать пайплайн предобработки"""

    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])

    return preprocessor


def create_model_pipeline(preprocessor, model):
    """Создать полный пайплайн: предобработка + модель"""

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    return pipeline


# ==============================================================================
# 3. КРОСС-ВАЛИДАЦИЯ
# ==============================================================================

def compare_cv_strategies(X, y, pipeline, groups=None):
    """Сравнить разные стратегии кросс-валидации"""

    cv_strategies = {
        'KFold': KFold(n_splits=5, shuffle=True, random_state=42),
        # TODO: добавьте TimeSeriesSplit если есть временная зависимость
        # 'TimeSeries': TimeSeriesSplit(n_splits=5),
        # TODO: добавьте GroupKFold если есть группы
        # 'GroupKFold': GroupKFold(n_splits=5)
    }

    results = {}

    for cv_name, cv_splitter in cv_strategies.items():
        scores = []

        # TODO: реализуйте цикл кросс-валидации
        # for train_idx, val_idx in cv_splitter.split(X, y, groups):
        #     X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        #     y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        #
        #     pipeline.fit(X_train_fold, y_train_fold)
        #     y_pred = pipeline.predict(X_val_fold)
        #     score = r2_score(y_val_fold, y_pred)
        #     scores.append(score)

        results[cv_name] = {
            'mean': None,  # TODO: np.mean(scores)
            'std': None    # TODO: np.std(scores)
        }

    return results


# ==============================================================================
# 4. BOOTSTRAP ДЛЯ ДОВЕРИТЕЛЬНЫХ ИНТЕРВАЛОВ
# ==============================================================================

def bootstrap_confidence_interval(X, y, pipeline, metric_func, n_bootstrap=1000, alpha=0.05):
    """Построить доверительный интервал метрики через bootstrap"""

    n_samples = len(X)
    bootstrap_metrics = []

    for i in range(n_bootstrap):
        # TODO: создайте bootstrap-выборку
        # indices = np.random.choice(n_samples, n_samples, replace=True)
        # X_boot = X.iloc[indices]
        # y_boot = y.iloc[indices]

        # TODO: обучите модель и вычислите метрику
        # pipeline.fit(X_boot, y_boot)
        # y_pred = pipeline.predict(X_boot)
        # metric = metric_func(y_boot, y_pred)
        # bootstrap_metrics.append(metric)

        pass

    # TODO: вычислите квантили для доверительного интервала
    lower = None  # np.percentile(bootstrap_metrics, 100 * alpha / 2)
    upper = None  # np.percentile(bootstrap_metrics, 100 * (1 - alpha / 2))

    return lower, upper, bootstrap_metrics


def compare_models_bootstrap(X, y, pipeline1, pipeline2, n_bootstrap=1000):
    """Сравнить две модели через bootstrap"""

    differences = []

    for i in range(n_bootstrap):
        # TODO: создайте bootstrap-выборку
        # indices = np.random.choice(len(X), len(X), replace=True)
        # X_boot = X.iloc[indices]
        # y_boot = y.iloc[indices]

        # TODO: обучите обе модели
        # pipeline1.fit(X_boot, y_boot)
        # pipeline2.fit(X_boot, y_boot)

        # TODO: вычислите разность метрик
        # y_pred1 = pipeline1.predict(X_boot)
        # y_pred2 = pipeline2.predict(X_boot)
        # diff = r2_score(y_boot, y_pred1) - r2_score(y_boot, y_pred2)
        # differences.append(diff)

        pass

    return differences


# ==============================================================================
# 5. PERMUTATION TEST
# ==============================================================================

def permutation_test(X, y, pipeline, n_permutations=1000):
    """Проверка значимости модели через permutation test"""

    # TODO: обучите модель на реальных данных
    # pipeline.fit(X, y)
    # y_pred = pipeline.predict(X)
    # true_score = r2_score(y, y_pred)

    permuted_scores = []

    for i in range(n_permutations):
        # TODO: перемешайте целевую переменную
        # y_perm = y.sample(frac=1, random_state=i).values

        # TODO: обучите модель на перемешанных данных
        # pipeline.fit(X, y_perm)
        # y_pred_perm = pipeline.predict(X)
        # perm_score = r2_score(y_perm, y_pred_perm)
        # permuted_scores.append(perm_score)

        pass

    # TODO: вычислите p-value
    # p_value = np.mean([s >= true_score for s in permuted_scores])

    return None, permuted_scores  # замените на true_score, permuted_scores


# ==============================================================================
# 6. FEATURE IMPORTANCE
# ==============================================================================

def calculate_permutation_importance(X, y, pipeline, n_repeats=10):
    """Вычислить permutation importance"""

    # TODO: обучите модель
    # pipeline.fit(X, y)

    # TODO: вычислите permutation importance
    # perm_importance = permutation_importance(
    #     pipeline, X, y,
    #     n_repeats=n_repeats,
    #     random_state=42,
    #     scoring='r2'
    # )

    # TODO: создайте DataFrame с результатами
    # importance_df = pd.DataFrame({
    #     'feature': X.columns,
    #     'importance_mean': perm_importance.importances_mean,
    #     'importance_std': perm_importance.importances_std
    # }).sort_values('importance_mean', ascending=False)

    return None  # замените на importance_df


def plot_feature_importance(importance_df, top_n=20):
    """Визуализировать важность признаков"""

    plt.figure(figsize=(10, 8))

    # TODO: постройте barplot для top_n признаков
    # top_features = importance_df.head(top_n)
    # plt.barh(top_features['feature'], top_features['importance_mean'])
    # plt.xlabel('Permutation Importance')
    # plt.title(f'Top {top_n} Features by Permutation Importance')

    plt.tight_layout()
    plt.show()


# ==============================================================================
# 7. SHAP АНАЛИЗ
# ==============================================================================

def calculate_shap_values(pipeline, X_sample):
    """Вычислить SHAP значения"""

    # TODO: извлеките обученную модель из пайплайна
    # model = pipeline.named_steps['model']

    # TODO: создайте SHAP explainer
    # explainer = shap.Explainer(model, X_sample)
    # shap_values = explainer(X_sample)

    return None  # замените на shap_values


def plot_shap_summary(shap_values, X_sample):
    """Визуализировать SHAP summary plot"""

    # TODO: постройте summary plot
    # shap.summary_plot(shap_values, X_sample)

    pass


def plot_shap_waterfall(shap_values, X_sample, observation_index=0):
    """Визуализировать SHAP waterfall для одного наблюдения"""

    # TODO: постройте waterfall plot
    # shap.waterfall_plot(shap_values[observation_index])

    pass


# ==============================================================================
# 8. ОСНОВНОЙ WORKFLOW
# ==============================================================================

def main():
    """Основной workflow лабораторной работы"""

    print("=" * 80)
    print("ЛАБОРАТОРНАЯ РАБОТА 3: СТАТИСТИЧЕСКИЙ ВЫВОД И ИНТЕРПРЕТАЦИЯ")
    print("=" * 80)

    # Шаг 1: Загрузка данных
    print("\n[1] Загрузка и подготовка данных...")
    X, y = load_and_prepare_data()

    # TODO: определите списки числовых и категориальных признаков
    # numeric_features = [...]
    # categorical_features = [...]

    # Шаг 2: Разделение на train/test
    print("\n[2] Разделение на train/test...")
    # X_train, X_test, y_train, y_test = train_test_split(
    #     X, y, test_size=0.2, random_state=42
    # )

    # Шаг 3: Создание пайплайнов
    print("\n[3] Создание пайплайнов...")
    # preprocessor = create_preprocessing_pipeline(numeric_features, categorical_features)

    # TODO: создайте несколько моделей для сравнения
    # pipeline_linear = create_model_pipeline(preprocessor, LinearRegression())
    # pipeline_rf = create_model_pipeline(preprocessor, RandomForestRegressor(random_state=42))
    # pipeline_gb = create_model_pipeline(preprocessor, GradientBoostingRegressor(random_state=42))

    # Шаг 4: Сравнение стратегий CV
    print("\n[4] Сравнение стратегий кросс-валидации...")
    # cv_results = compare_cv_strategies(X_train, y_train, pipeline_rf)
    # print(cv_results)

    # Шаг 5: Bootstrap доверительные интервалы
    print("\n[5] Bootstrap для доверительных интервалов...")
    # lower, upper, boot_metrics = bootstrap_confidence_interval(
    #     X_train, y_train, pipeline_rf, r2_score, n_bootstrap=1000
    # )
    # print(f"95% CI для R²: [{lower:.4f}, {upper:.4f}]")

    # Шаг 6: Сравнение моделей через bootstrap
    print("\n[6] Сравнение моделей через bootstrap...")
    # differences = compare_models_bootstrap(
    #     X_train, y_train, pipeline_rf, pipeline_linear, n_bootstrap=1000
    # )

    # Шаг 7: Permutation test
    print("\n[7] Permutation test...")
    # true_score, perm_scores = permutation_test(
    #     X_train, y_train, pipeline_rf, n_permutations=1000
    # )

    # Шаг 8: Обучение финальных моделей
    print("\n[8] Обучение финальных моделей на всех train данных...")
    # pipeline_rf.fit(X_train, y_train)
    # y_pred_test = pipeline_rf.predict(X_test)
    # test_r2 = r2_score(y_test, y_pred_test)
    # print(f"Test R²: {test_r2:.4f}")

    # Шаг 9: Permutation Importance
    print("\n[9] Вычисление permutation importance...")
    # importance_df = calculate_permutation_importance(
    #     X_test, y_test, pipeline_rf, n_repeats=10
    # )
    # plot_feature_importance(importance_df, top_n=20)

    # Шаг 10: SHAP анализ
    print("\n[10] SHAP анализ...")
    # X_sample = X_test.sample(min(1000, len(X_test)), random_state=42)
    # X_sample_transformed = pipeline_rf.named_steps['preprocessor'].transform(X_sample)
    # shap_values = calculate_shap_values(pipeline_rf, X_sample_transformed)
    # plot_shap_summary(shap_values, X_sample)

    print("\n" + "=" * 80)
    print("ГОТОВО!")
    print("=" * 80)


if __name__ == "__main__":
    main()

In [1]:
import pandas as pd

df = pd.read_parquet("yellow_tripdata_2025-08.parquet")

display(df.head())
print("Размер (строки, столбцы):", df.shape)
print("\nСписок колонок:")
print(df.columns)
print("\nТипы данных:")
print(df.dtypes)
print("\nПропущенные значения:")
print(df.isna().sum())


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,2,2025-08-01 00:52:23,2025-08-01 01:12:20,1.0,8.44,1.0,N,138,141,1,33.8,6.0,0.5,5.00,6.94,1.0,57.49,2.5,1.75,0.00
1,2,2025-08-01 00:03:01,2025-08-01 00:15:33,2.0,4.98,1.0,N,138,193,1,21.2,6.0,0.5,0.00,0.00,1.0,30.45,0.0,1.75,0.00
2,7,2025-08-01 00:24:38,2025-08-01 00:24:38,2.0,1.89,1.0,N,249,45,1,14.2,0.0,0.5,3.99,0.00,1.0,23.94,2.5,0.00,0.75
3,7,2025-08-01 00:48:19,2025-08-01 00:48:19,1.0,2.35,1.0,N,79,229,1,11.4,0.0,0.5,3.43,0.00,1.0,20.58,2.5,0.00,0.75
4,2,2025-08-01 00:25:34,2025-08-01 00:33:18,1.0,2.14,1.0,N,43,48,1,11.4,1.0,0.5,2.57,0.00,1.0,19.72,2.5,0.00,0.75


Размер (строки, столбцы): (3574091, 20)

Список колонок:
Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee',
       'cbd_congestion_fee'],
      dtype='object')

Типы данных:
VendorID                          int32
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int32
DOLocationID                      int32
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         floa